In [ ]:
from  nano_graphrag import GraphRAG, QueryParam 
import nest_asyncio
nest_asyncio.apply()
from sentence_transformers import SentenceTransformer
import numpy as np
from nano_graphrag import GraphRAG



# llama 활용

In [18]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수에서 Hugging Face 토큰 가져오기
hf_token = os.getenv("HUGGINGFACE_API_KEY")


In [3]:


model = SentenceTransformer("all-MiniLM-L6-v2")

# nano-graphrag에서 요구하는 래핑
def wrap_embedding_func_with_attrs(embedding_dim, max_token_size):
    def decorator(func):
        func.embedding_dim = embedding_dim
        func.max_token_size = max_token_size
        return func
    return decorator

@wrap_embedding_func_with_attrs(embedding_dim=384, max_token_size=512)
async def local_embedding_func(texts: list[str]) -> np.ndarray:
    return model.encode(texts, convert_to_numpy=True)

async def my_custom_llm(prompt, system_prompt=None, history_messages=[], **kwargs):
    response = await ollama_complete_if_cache(
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        model= 'llama3',
        **kwargs
    )
    print("🧠 LLaMA3 응답:", response)
    return response

# ✅ GraphRAG 객체 생성
graph_func = GraphRAG(
    working_dir="./llama_graph_data",
    best_model_func=my_custom_llm,             # 👉 Ollama 기반 LLM 사용
    embedding_func=local_embedding_func        # 👉 SentenceTransformer 기반 로컬 임베딩
)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
/Users/mac/AIworkspace/torchspace/nano-graphrag/nano_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:nano-graphrag:Load KV full_docs with 0 data
INFO:nano-graphrag:Load KV text_chunks with 0 data
INFO:nano-graphrag:Load KV llm_response_cache with 8 data
INFO:nano-graphrag:Load KV community_reports with 0 data
INFO:nano-graphrag:Loaded graph from ./llama_graph_data/graph_chunk_entity_relation.graphml with 0 nodes, 0 edges
INFO:nano-vectordb:Load (0, 384) data
INFO:nano-vectordb:Init {'embedding_dim': 384, 'metric': 'cosine', 'storage_file': './llama_graph_data/vdb_entities.json'} 0 da

In [4]:

# [3] 문서 삽입 (한 번만 실행하면 저장됨)
with open("./book.txt") as f:
    text = f.read()
    graph_func.insert(text)  # 자동으로 chunking, graph 구성


INFO:nano-graphrag:[New Docs] inserting 1 docs
INFO:nano-graphrag:[New Chunks] inserting 4 chunks
INFO:nano-graphrag:[Entity Extraction]...


📣 [ollama_complete] 함수 호출됨!!
📝 Full prompt length (chars): 6038
📣 [ollama_complete] 함수 호출됨!!
📝 Full prompt length (chars): 5943
📣 [ollama_complete] 함수 호출됨!!
📝 Full prompt length (chars): 5797
📣 [ollama_complete] 함수 호출됨!!
📝 Full prompt length (chars): 1756


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


🧠 LLaMA3 응답: Here is the output in the required format:

## ("entity"<|>Master Scrooge<|>person<|>He was a bitter old man with a heart full of coldness and stone. He had no love for Christmas or its traditions, and he spent most of his time alone, away from his nephew's family.)
## ("entity"<|>Schoolmaster<|>person<|>He was the master of a school where children were forced to learn. He was a stern man who liked to keep people in line.)
## ("entity"<|>Ghost<|>person<|>She was a ghostly figure who appeared to Scrooge and took him on a journey through time, showing him the errors of his ways and trying to make him see the error of his ways.)
## ("entity"<|>Fezziwig<|>organization<|>He was an old gentleman who used to be a kind and generous employer. He loved Christmas and liked to celebrate it with his employees.)
## ("relationship"<|>Master Scrooge<|>Schoolmaster<|>The schoolmaster shook hands with Master Scrooge, which made him nervous and uneasy in his mind.<|>5)
## ("relationship"<|>M

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


...

ReadTimeout: 

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


🧠 LLaMA3 응답: ## ("entity"<|>Ebenezer Scrooge<|>person<|>He was a bitter old man with a heart full of coldness and stone. He had no love for Christmas or its traditions, and he spent most of his time alone, away from his nephew's family.)
## ("entity"<|>Nephew<|>person<|>He was Scrooge's nephew, who lived with his wife and children, celebrating Christmas in a warm and loving manner.)
## ("entity"<|>Sister<|>person<|>She was Scrooge's sister, who was kind and gentle, but also had a sense of mischief and playfulness.)
## ("entity"<|>Postboy<|>person<|>He was a tired and worn-out postboy, who had been working all day and just wanted to rest and enjoy the Christmas celebrations.)
## ("entity"<|>Dick Wilkins<|>person<|>He was Scrooge's fellow-'prentice, who was young and full of life, enjoying the festive atmosphere and celebrating with his friends.)
## ("entity"<|>Warehouse<|>organization<|>It was a large warehouse where Fezziwig held Christmas celebrations for his employees, filled with mu

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


🧠 LLaMA3 응답: Here are the additional entities:

## "entity"<|>"Ghost of Christmas Past"<|>event<|>"A spirit that appears to Scrooge and guides him on a journey through his past"<|>

## "entity"<|>"Ghost of Christmas Present"<|>event<|>"A spirit that appears to Scrooge and guides him on a journey through his present, revealing the struggles of those around him"<|>

## "entity"<|>"Ghost of Christmas Yet to Come"<|>event<|>"A spirit that appears to Scrooge and guides him on a journey through his future, showing him a possible outcome if he does not change his ways"<|>

## "entity"<|>"Jacob Marley's Ghost"<|>event<|>"The ghost of Jacob Marley, Scrooge's former business partner who died in a state of despair due to their greedy and selfish ways"<|>

## "entity"<|>"Bob Cratchit's Family"<|>person<|>"The family of Bob Cratchit, including his wife and children, who are struggling to make ends meet despite Scrooge's mistreatment of them"<|>

## "entity"<|>"Mansion"<|>geo<|>"A large house that w

# 엑사원 활용 

In [1]:
import httpx
import numpy as np
from sentence_transformers import SentenceTransformer
from nano_graphrag import GraphRAG

# ✅ EXAONE 비동기 LLM 함수
async def exaone_generate(prompt, system_prompt=None, history_messages=[], **kwargs):
    async with httpx.AsyncClient(timeout=180.0) as client:
        response = await client.post(
            "http://localhost:8000/generate",  # 서버 주소 맞게 수정
            json={"prompt": prompt}
        )
        return response.json()["result"]




/Users/mac/AIworkspace/torchspace/nano-graphrag/nano_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ✅ SentenceTransformer 로드
from sentence_transformers import SentenceTransformer

# 안전하게 잘 작동하는 모델
model = SentenceTransformer("jhgan/ko-sroberta-multitask")


# ✅ nano-graphrag에 맞는 embedding function wrapper
def wrap_embedding_func_with_attrs(embedding_dim, max_token_size):
    def decorator(func):
        func.embedding_dim = embedding_dim
        func.max_token_size = max_token_size
        return func
    return decorator

@wrap_embedding_func_with_attrs(embedding_dim=384, max_token_size=512)
async def local_embedding_func(texts: list[str]) -> np.ndarray:
    return model.encode(texts, convert_to_numpy=True)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: jhgan/ko-sroberta-multitask


In [8]:
# ✅ GraphRAG 객체 생성
graph_func = GraphRAG(
    working_dir="./my_graph_kor_exaone",
    best_model_func=exaone_generate,  # ⬅️ 여기만 바뀐 포인트!
    embedding_func=local_embedding_func,
    chunk_token_size=100000,
    chunk_overlap_token_size=0,
)

INFO:nano-graphrag:Creating working directory ./my_graph_kor_exaone
INFO:nano-graphrag:Load KV full_docs with 0 data
INFO:nano-graphrag:Load KV text_chunks with 0 data
INFO:nano-graphrag:Load KV llm_response_cache with 0 data
INFO:nano-graphrag:Load KV community_reports with 0 data
INFO:nano-vectordb:Init {'embedding_dim': 384, 'metric': 'cosine', 'storage_file': './my_graph_kor_exaone/vdb_entities.json'} 0 data


In [10]:
import pandas as pd
# 블로그 데이터 불러오기
df = pd.read_csv('hair_data/blog_allure.csv')

docs = df["본문"].dropna().astype(str).tolist()
await graph_func.ainsert(docs[:10])  # ✅ 안전하게 실행됨



INFO:nano-graphrag:[New Docs] inserting 10 docs
INFO:nano-graphrag:[New Chunks] inserting 10 chunks
INFO:nano-graphrag:[Entity Extraction]...
INFO:nano-graphrag:Writing graph with 0 nodes, 0 edges


CancelledError: 

In [14]:
import httpx

# EXAONE API 호출 함수
async def exaone_generate(prompt):
    async with httpx.AsyncClient(timeout=60.0) as client:
        res = await client.post(
            "http://localhost:11434/api/generate",
            json={
                "model": "exaone3.5:7.8b",
                "prompt": prompt,
                "stream": False
            }
        )
        return res.json()["response"]

# GraphRAG에서 사용할 LLM 함수
async def my_custom_llm(prompt, system_prompt=None, history_messages=[], **kwargs):
    full_prompt = ""
    if system_prompt:
        full_prompt += f"[System]\n{system_prompt}\n\n"
    for message in history_messages:
        role = message.get("role", "user")
        content = message.get("content", "")
        full_prompt += f"[{role.capitalize()}]\n{content}\n\n"
    full_prompt += f"[User]\n{prompt}"

    return await exaone_generate(full_prompt)


'-목표-\n다음에 제시된 텍스트 문서와 엔티티 타입 목록을 바탕으로, 해당 타입에 해당하는 엔티티들을 모두 식별하고 이들 간의 명확한 관계를 추출하세요.\n\n-단계-\n1. 엔티티 식별:\n   문서 내에서 지정된 엔티티 타입({entity_types})에 해당하는 엔티티를 모두 찾아내고, 각각에 대해 다음 정보를 추출합니다:\n   - entity_name: 엔티티의 이름 (모두 대문자로 표기)\n   - entity_type: 지정된 엔티티 타입 중 하나\n   - entity_description: 해당 엔티티의 특성과 활동에 대한 상세 설명\n\n   아래 형식으로 출력합니다:\n   ("entity"<|><엔티티_이름><|><엔티티_타입><|><엔티티_설명>)\n\n2. 관계 추출:\n   위에서 식별한 엔티티들 중 명확히 관련된 엔티티 쌍(source_entity, target_entity)을 찾고, 각각에 대해 다음 정보를 추출합니다:\n   - source_entity: 관계의 출발점이 되는 엔티티 이름\n   - target_entity: 관계의 도착점이 되는 엔티티 이름\n   - relationship_description: 두 엔티티가 관련 있다고 판단한 이유를 설명하는 문장\n   - relationship_strength: 관계의 강도를 나타내는 숫자 (1–10 사이)\n\n   아래 형식으로 출력합니다:\n   ("relationship"<|><출발_엔티티><|><도착_엔티티><|><관계_설명><|><관계_강도>)\n\n3. 출력:\n   엔티티와 관계 정보를 모두 **하나의 목록**으로 묶어 출력하며, 각 항목은 **{record_delimiter}** 로 구분합니다.\n   출력의 마지막은 반드시 {completion_delimiter} 로 마무리해야 합니다.\n\n######################\n-예시-\n######################\nEntity_types: [person, technology, mi

In [ ]:
from nano_graphrag import GraphRAG
import pandas as pd

# GraphRAG 인스턴스 생성
graph_func = GraphRAG(
    working_dir="./my_graph_kor_exaone",
    best_model_func=my_custom_llm,         # ✅ EXAONE 기반 LLM
    embedding_func=local_embedding_func,   # ✅ SentenceTransformer 임베딩
    chunk_token_size=100000,
    chunk_overlap_token_size=0,
)

# 블로그 데이터 불러오기
df = pd.read_csv('hair_data/blog_allure.csv')
docs = df["본문"].dropna().astype(str).tolist()

# 문서 삽입
graph_func.insert(docs[:10])


In [20]:
import httpx
from nano_graphrag.prompt_kr import PROMPTS

# 비동기 EXAONE 호출 함수
async def exaone_generate(prompt):
    async with httpx.AsyncClient(timeout=180.0) as client:  # 60초로 늘림
        res = await client.post(
            "http://localhost:11434/api/generate",
            json={
                "model": "exaone3.5:7.8b",
                "prompt": prompt,
                "stream": False
            }
        )
        return res.json()["response"]


# 프롬프트 기반 엔티티 및 관계 추출 함수
async def extract_entities_and_relationships(input_text, entity_types):
    prompt_template = PROMPTS["entity_extraction"]

    # 구성요소
    record_delimiter = "<R>"
    completion_delimiter = "<END>"

    # 포맷팅된 프롬프트 생성
    formatted_prompt = prompt_template.format(
        entity_types=entity_types,
        input_text=input_text,
        record_delimiter=record_delimiter,
        completion_delimiter=completion_delimiter
    )

    # EXAONE에 전달하고 결과 받기
    output = await exaone_generate(formatted_prompt)

    # 결과 파싱은 필요 시 추가
    return output


In [21]:
import asyncio

# 예시 입력
text = """
OpenAI는 대규모 언어 모델을 개발하는 인공지능 연구소로, GPT 시리즈를 통해 생성형 AI의 한계를 확장하고 있다.
2023년에는 Microsoft와의 협력을 통해 Azure에 AI 서비스를 배포하기도 했다.
"""
entity_types = ["organization", "technology", "year"]

# 비동기 실행
result = asyncio.run(extract_entities_and_relationships(text, entity_types))
print(result)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


<R>("entity"<|>"OPENAI"<|>"ORGANIZATION"<|>"OpenAI는 대규모 언어 모델 개발을 주도하는 인공지능 연구소이며, GPT 시리즈를 통해 생성형 AI 분야의 혁신을 이끌고 있습니다.")<R>
<R>("entity"<|>"GPT SERIES"<|>"TECHNOLOGY"<|>"GPT 시리즈는 OpenAI가 개발한 고급 언어 모델로서 생성형 AI 기술의 발전에 중추적인 역할을 합니다.")<R>
<R>("entity"<|>"AZURE"<|>"ORGANIZATION"<|>"Microsoft의 클라우드 플랫폼으로, 여기서 OpenAI의 AI 기술이 배포되어 기업과 개발자들에게 제공됩니다.")<R>
<R>("relationship"<|>"OPENAI"<|>"GPT SERIES"<|>"OpenAI는 GPT 시리즈 기술을 통해 생성형 AI 분야에서 혁신을 주도하며 핵심 기술 역할을 합니다."<R>9)<R>
<R>("relationship"<|>"OPENAI"<|>"AZURE"<|>"OpenAI는 Microsoft의 Azure 플랫폼을 통해 기술 서비스를 배포함으로써 파트너십을 통해 시장 접근성을 확대하고 있습니다."<R>8)<R>
<R>("relationship"<|>"GPT SERIES"<|>"AZURE"<|>"GPT 시리즈 기술은 Azure를 통해 클라우드 기반으로 제공되며, 이를 통해 사용자들에게 안정적이고 확장 가능한 AI 솔루션을 제공합니다."<R>8)<R>
<R>("entity"<|>"2023"<|>"YEAR"<|>"2023년은 OpenAI와 Microsoft 간의 중요한 협력이 이루어진 해로, 특히 Azure를 통한 AI 서비스 배포가 이루어졌습니다.")<R>
<R>("relationship"<|>"OPENAI"<|>"2023"<|>"2023년의 사건은 OpenAI의 기술적 발전과 시장 진출 전략이 본격적으로 진행된 해를 나타냅니다."<R>7)<R>
<R>("relationship"<|>"AZURE"<|>"2023"<|>"2023년

# GPT 활용
- 한국어 블로그 크롤링한 것을 document로 활용하여 graphrag에 적용
- gpt-4o-mini를 활용할 예정이며, prompt를 한국어로 바꾸는게 필요함 

### 코드 구현
1. 전체 코드 흐름에 대해 모듈화 되어 있는 것을 내 방식대로 바꾸기 
    - 이를 위해 전체 코드에 대한 이해가 필요함

2. gpt-4o-mini만 사용할 예정이며 prompt를 한국어로 바꿈 
    - 이때, 우리의 문서에 task에 맞게 prompt를 조정하는 과정이 필요함 
    - 한국어 document를 전처리하는 과정도 필요함 

In [26]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일에 OPENAI_API_KEY가 포함되어 있어야 함

True

In [27]:
from nano_graphrag._llm import openai_complete_if_cache

async def my_custom_llm(prompt, system_prompt=None, history_messages=[], **kwargs):
    return await openai_complete_if_cache(
        "gpt-4o-mini",  # 원하는 모델
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        **kwargs
    )


In [28]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 안정적으로 로딩 가능한 한국어 SBERT 모델
model_name = 'jhgan/ko-sroberta-multitask'
model = SentenceTransformer(model_name)

# nano-graphrag에서 요구하는 래핑
def wrap_embedding_func_with_attrs(embedding_dim, max_token_size):
    def decorator(func):
        func.embedding_dim = embedding_dim
        func.max_token_size = max_token_size
        return func
    return decorator

@wrap_embedding_func_with_attrs(embedding_dim=768, max_token_size=512)
async def local_embedding_func(texts: list[str]) -> np.ndarray:
    """
    한국어 문장 리스트를 입력받아 SBERT 임베딩을 반환합니다.
    """
    return model.encode(texts, convert_to_numpy=True)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: jhgan/ko-sroberta-multitask
/Users/mac/AIworkspace/torchspace/nano-graphrag/nano_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
from nano_graphrag import GraphRAG

# graph_func = GraphRAG(
#     working_dir="./my_graphrag_data",
#     best_model_func=my_custom_llm , # 여기서 지정
#     embedding_func=local_embedding_func,  # 여기서 지정
# )
graph_func = GraphRAG(
    working_dir="./my_graph_kor",
    best_model_func=my_custom_llm,
    embedding_func=local_embedding_func,

    # ✅ 청크 사이즈를 아주 크게 설정해서 "절대 자르지 않게" 함
    chunk_token_size=100000,
    chunk_overlap_token_size=0,
)

INFO:nano-graphrag:Creating working directory ./my_graph_kor
INFO:nano-graphrag:Load KV full_docs with 0 data
INFO:nano-graphrag:Load KV text_chunks with 0 data
INFO:nano-graphrag:Load KV llm_response_cache with 0 data
INFO:nano-graphrag:Load KV community_reports with 0 data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './my_graph_kor/vdb_entities.json'} 0 data


In [31]:
import pandas as pd

import pandas as pd 
df= pd.read_csv('hair_data/naver_blog_per3.csv')
docs = df["요약"].dropna().astype(str).tolist()  # 🔥 행 하나 = 하나의 청크로 사용됨
graph_func.insert(docs[:2])

INFO:nano-graphrag:[New Docs] inserting 2 docs
INFO:nano-graphrag:[New Chunks] inserting 2 chunks
INFO:nano-graphrag:[Entity Extraction]...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:nano-graphrag:Inserting 10 vectors to entities


Batches: 100%|██████████| 1/1 [00:00<00:00,  9.14it/s]
INFO:nano-graphrag:[Community Report]...
INFO:nano-graphrag:Writing graph with 10 nodes, 0 edges


EmptyNetworkError: EmptyNetworkError

In [25]:
# Perform global graphrag search
print(graph_func.query("나는 달걀형이고, 봄에 잘 어울리는 헤어스타일을 추천해줘"))

# Perform local graphrag search (I think is better and more scalable one)
# print(graph_func.query("What are the top themes in this story?", param=QueryParam(mode="local")))

INFO:nano-graphrag:Revtrieved 6 communities
INFO:nano-graphrag:Grouping to 1 groups for global search
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## 봄에 어울리는 헤어스타일 추천

당신이 달걀형 얼굴이라면, 봄 시즌에 잘 어울릴 여러 가지 헤어스타일이 있습니다. 아래에서 두 가지 스타일과 함께 그 특징을 살펴보겠습니다.

### 1. 깻잎 머리 스타일

현재 깻잎 머리 스타일이 큰 인기를 얻고 있습니다. 이 스타일은 우아함을 강조하며, 봄의 계절적 미와 잘 맞아떨어집니다. 달걀형 얼굴에 적합한 이 스타일은 얼굴의 윤곽을 부각시키고 자연스러운 매력을 더할 수 있습니다. 특히 봄에는 이 스타일이 한층 더 빛을 발할 것입니다.

### 2. 히메 컷

히메 컷 또한 추천해 드릴 만한 스타일입니다. 이 컷은 긴 슬릭 뱅과 뒤로 흐르는 긴 머리가 특징입니다. 다양한 얼굴형에 잘 어울리는 이 스타일은 특히 달걀형 얼굴에 flattering한 선택이 될 수 있습니다. 봄 패션과 잘 어우러져 인기가 상승할 것으로 예상되므로, 시도해 볼 만한 가치가 있습니다.

### 3. 액세서리 추가

헤어스타일을 한층 더 빛나게 하려면 액세서리의 역할도 중요합니다. 스프링 시즌에 잘 어울리는 헤어 클립이나 베레모 같은 액세서리를 추가하면, 스타일에 매력을 더하고 봄의 경쾌한 느낌을 강화할 수 있습니다. 

결론적으로, 달걀형 얼굴에는 깻잎 머리와 히메 컷이 잘 어울리며, 다양한 액세서리를 활용하면 봄의 매력을 더욱 살릴 수 있습니다. 이러한 스타일을 통해 당신의 개성을 효과적으로 표현할 수 있기를 바랍니다.
